In [1]:
import pandas as pd
import numpy as np

### Energy Indicators DF
Source: http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls

In [2]:
energy = pd.read_excel("Energy Indicators.xls", skip_footer=38, skiprows=17,index_col = 1)
energy.drop(energy.columns[[0,1]], axis=1, inplace=True)
energy.rename(columns={'Unnamed: 2':'Country','Petajoules':'Energy Supply',
                       'Gigajoules':'Energy Supply per Capita',"%":"% Renewable's"}, inplace=True)

energy['Energy Supply'].replace(to_replace="...",value=np.nan, inplace=True)
energy['Energy Supply per Capita'].replace(to_replace="...",value=np.nan, inplace=True)
energy.index.names = ["Country"]
energy['Energy Supply'] = energy['Energy Supply'] * 1000000

countriesdict = {"Republic of Korea": "South Korea","United States of America": "United States",
              "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
              "China, Hong Kong Special Administrative Region": "Hong Kong"}

energy.rename(countriesdict, inplace=True)

energy.head()

,Energy Supply,Energy Supply per Capita,% Renewable's
Country,,,
Afghanistan,3.210000e+08,10.0,78.669280
Albania,1.020000e+08,35.0,100.000000
Algeria,1.959000e+09,51.0,0.551010
American Samoa,NaN,NaN,0.641026
Andorra,9.000000e+06,121.0,88.695650


In [3]:
correct_country_names = {}
for country in energy.index:
    if " (" in country:
        correct_name = country.split(" (")[0]
        correct_country_names[country] = correct_name
correct_country_names

{u'Bolivia (Plurinational State of)': u'Bolivia',
 u'Falkland Islands (Malvinas)': u'Falkland Islands',
 u'Iran (Islamic Republic of)': u'Iran',
 u'Micronesia (Federated States of)': u'Micronesia',
 u'Sint Maarten (Dutch part)': u'Sint Maarten',
 u'Venezuela (Bolivarian Republic of)': u'Venezuela'}

In [4]:
energy.rename(correct_country_names, inplace=True)

### GDP DF
Source: http://data.worldbank.org/indicator/NY.GDP.MKTP.CD

In [5]:
GDP = pd.read_csv('world_bank.csv', skiprows=4)

countriesdict = {"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran",
                 "Hong Kong SAR, China": "Hong Kong"}

GDP.set_index('Country Name', inplace=True)
GDP.rename(index=countriesdict, inplace=True)
GDP = GDP[['2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']]

GDP.head(2)

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country Name,,,,,,,,,,
Aruba,2.421475e+09,2.623726e+09,2.791961e+09,2.498933e+09,2.467704e+09,2.584464e+09,NaN,NaN,NaN,NaN
Andorra,3.536633e+09,4.010991e+09,4.001201e+09,3.650083e+09,3.346517e+09,3.427023e+09,3.146152e+09,3.248925e+09,NaN,NaN


### ScimEn DF
Source: http://www.scimagojr.com/countryrank.php?category=2102

In [6]:
ScimEn = pd.read_excel('scimagojr-3.xlsx')

ScimEn.set_index('Rank', inplace=True)

mask = ScimEn.index<=15
ScimEn = ScimEn[mask]

ScimEn

,Country,Documents,Citable documents,Citations,Self-citations,Citations per document,H index
Rank,,,,,,,
1,China,127050,126767,597237,411683,4.70,138
2,United States,96661,94747,792274,265436,8.20,230
3,Japan,30504,30287,223024,61554,7.31,134
4,United Kingdom,20944,20357,206091,37874,9.84,139
5,Russian Federation,18534,18301,34266,12422,1.85,57
6,Canada,17899,17620,215003,40930,12.01,149
7,Germany,17027,16831,140566,27426,8.26,126
8,India,15005,14841,128763,37209,8.58,115
9,France,13153,12973,130632,28601,9.93,114


### Merging all 3 DFs

In [22]:
#first 2 (energy & GDP)
eg = pd.merge(energy, GDP, how='inner', left_index=True, right_index=True)

#adding the 3rd one
egs = pd.merge(eg, ScimEn, how='inner', left_index=True, right_on='Country')
egs['Rank'] = egs.index
egs.set_index('Country', inplace=True)

#Reindexing the columns
egs = egs.ix[:,['Rank',
                'Documents',
                'Citable documents',
                'Citations',
                'Self-citations',
                'Citations per document',
                'H index', 'Energy Supply',
                'Energy Supply per Capita',
                '% Renewable',
                '2006',
                '2007',
                '2008',
                '2009',
                '2010',
                '2011',
                '2012',
                '2013',
                '2014',
                '2015']]

egs['avgGDP'] = np.mean(egs[[str(x) for x in range(2006,2016)]], axis=1)
egs

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,avgGDP
Country,,,,,,,,,,,,,,,,,,,,,
Australia,14,8831,8725,90765,15606,10.28,107,5.386000e+09,231.0,NaN,...,8.530533e+11,1.054558e+12,9.265638e+11,1.142251e+12,1.389919e+12,1.537478e+12,1.563951e+12,1.454675e+12,1.339141e+12,1.200847e+12
Brazil,15,8668,8596,60702,14396,7.00,86,1.214900e+10,59.0,NaN,...,1.397084e+12,1.695825e+12,1.667020e+12,2.208872e+12,2.614573e+12,2.460658e+12,2.465774e+12,2.417046e+12,1.774725e+12,1.980922e+12
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296.0,NaN,...,1.464977e+12,1.549131e+12,1.371153e+12,1.613406e+12,1.788703e+12,1.824289e+12,1.837443e+12,1.783776e+12,1.550537e+12,1.609883e+12
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,NaN,...,3.552183e+12,4.598205e+12,5.109954e+12,6.100620e+12,7.572554e+12,8.560547e+12,9.607224e+12,1.048237e+13,1.100772e+13,6.934351e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165.0,NaN,...,3.439953e+12,3.752366e+12,3.418005e+12,3.417298e+12,3.757465e+12,3.543984e+12,3.752514e+12,3.879277e+12,3.363447e+12,3.532675e+12
Spain,12,9428,9330,123336,23964,13.08,115,4.923000e+09,106.0,NaN,...,1.479342e+12,1.634989e+12,1.499075e+12,1.431673e+12,1.487925e+12,1.339947e+12,1.369262e+12,1.381342e+12,1.199057e+12,1.408716e+12
France,9,13153,12973,130632,28601,9.93,114,1.059700e+10,166.0,NaN,...,2.663113e+12,2.923466e+12,2.693827e+12,2.646995e+12,2.862502e+12,2.681416e+12,2.808511e+12,2.839162e+12,2.418836e+12,2.686284e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+09,124.0,NaN,...,3.063005e+12,2.875463e+12,2.367127e+12,2.429603e+12,2.608995e+12,2.646003e+12,2.719509e+12,2.998834e+12,2.858003e+12,2.724482e+12
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26.0,NaN,...,1.201072e+12,1.186913e+12,1.323896e+12,1.656562e+12,1.822990e+12,1.828985e+12,1.863208e+12,2.042439e+12,2.095398e+12,1.597058e+12
